## Inferência

#### Teste estatístico de hipóteses

In [2]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Returns mu and sigma corresponding to a Binomial(n, p)"""
    
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    
    return mu, sigma

In [3]:
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

# The normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

In [4]:
# It's above the threshold if it's not below the threshold
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    """The probability that a N (mu, sigma) is greater than lo."""
    
    return 1 - normal_cdf(lo, mu, sigma)

In [5]:
# It's between if it's less than hi, but not less than lo
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that N(mu, sigma) is between lo and hi"""
    
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

In [6]:
# It's outside if it's not between
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that N(mu, sigma) is not between lo and hi"""
    
    return 1 - normal_probability_between(lo, hi, mu, sigma)

We can also do the opposite and find the region outside the threshold or the interval (symmetric) around the mean that is associated with a given level of probability.

In [7]:
def inversal_normal_cdf(p: float,
                        mu: float = 0,
                        sigma: float = 1,
                        tolerance: float = 0.00001) -> float:
    """Find approximate inverse using binary search"""
    
    if mu != 0 or sigma != 1:
        return mu + sigma * inversal_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0          # normal_cdf(-10) is (very close to) 0
    hi_z = 10.0            # normal_cdf(10) is (very close to) 1
    
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)
        
        if mid_p < p:
            low_z = mid_z
        else:
            hi_z = mid_z
    
    return mid_z

In [8]:
def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z <= z) = probability"""
    
    return inversal_normal_cdf(probability, mu, sigma)

In [9]:
def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z >= z) = probability"""
    
    return inversal_normal_cdf(1 - probability, mu, sigma)

In [10]:
def normal_two_sided_bounds(probability: float, 
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
    """Returns the symetric (about the man) bounds that contain
    that especified probability"""
    
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [11]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

assert mu_0 == 500
assert 15.8 < sigma_0 < 15.9

In [12]:
# significância (469, 531)

lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

assert 468.5 < lower_bound < 469.5
assert 530.5 < upper_bound < 531.5

In [13]:
# p = 0.5

lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# um erro tipo 2 indica a rejeição da hipotese nula

type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

power = 1 - type_2_probability

assert 0.886 < power < 0.888

print(power)

0.8865480012953671


In [14]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)

power = 1 - type_2_probability 

assert 526 < hi < 526.1
assert 0.9363 < power < 0.9364

print(hi, power)

526.0073585242053 0.9363794803307173


#### p-Values

Em vez de escolher limites com base em um limiar de probabilidade, é possível computar a probabilidade de observar um valor pelo menos tão extremo quanto o que já observamos de fato.

In [15]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """How likely are we to see a value at least as extreme as x
    (in either ditection) if out values are from a N(mu, sigma)?"""
    
    if x >= mu:
        # x is greater than the mean, so the tail is everything greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the man, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)

In [17]:
# we use 529.5 and not 530 because of continuity correction (the book has an explanation link)

two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [30]:
# simulation

import random

extreme_value_count = 0

for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                    for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1

assert 59 < extreme_value_count < 65

print(f"{extreme_value_count}")

62


In [31]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [33]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [35]:
# 525 C

upper_p_value(524.5, mu_0, sigma_0) # 0.06062885772582072

0.06062885772582072

In [39]:
upper_p_value(526.5, mu_0, sigma_0) # 0.04686839508859242

0.04686839508859242

#### Intervalos de confiança

In [40]:
# for 525 C

p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

In [43]:
# a moeda não será viciada se, assumindo que a probabilidade é 0.5 estar contida no intervalo de confiança

normal_two_sided_bounds(0.95, mu, sigma) # (0.4940490278129096, 0.5559509721870904)

(0.4940490278129096, 0.5559509721870904)

In [46]:
# for 540 C

p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

normal_two_sided_bounds(0.95, mu, sigma) # (0.5091095927295919, 0.5708904072704082)

# 0.5 está fora do intervalo gerado observando 540x cara na moeda, logo é uma moeda viciada

(0.5091095927295919, 0.5708904072704082)

#### p-haking

In [52]:
from typing import List

def run_experiment() -> List[bool]:
    """Flips a fit coin 1000 times, True = Heads, False = Taills"""
    
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """Using the 5% significance levels"""
    
    num_heads = len([flip for flip in experiment if flip])
    
    return num_heads < 469 or num_heads > 531

In [54]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

assert num_rejections == 46

#### Executando um teste A/B

In [55]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    
    return p, sigma

def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [57]:
# A is 200 click in 1000 view and B is 180 clicks in 1000 views

z = a_b_test_statistic(1000, 200, 1000, 180)
print(z)

-1.1403464899034472


In [62]:
two_sided_p_value(z)

#assert 0.253 < two_sided_p_value(z) < 0.255

0.254141976542236

In [63]:
# if B 150 clicks in 1000 views

z = a_b_test_statistic(1000, 200, 1000, 150)
print(z)

-2.948839123097944


In [65]:
# is big difference

two_sided_p_value(z)

0.003189699706216853

#### inferência Bayesiana

In [66]:
def B(alpha: float, beta: float) -> float:
    """A normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:          # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)